In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/P185/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [8]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [9]:
df["Region"] = regions

In [10]:
df.insert(0, "channel", df.pop("channel"))

In [11]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-100,2056.65,2832.29,3080.09,1586.98,2508.000,3006.21,9
1,-99,2058.87,2826.22,3086.70,1587.28,2499.280,3010.78,9
2,-98,2060.55,2820.90,3092.92,1587.57,2490.550,3015.35,9
3,-97,2061.83,2815.93,3098.94,1587.86,2481.830,3019.92,9
4,-96,2062.75,2811.14,3104.94,1588.16,2473.110,3024.49,9
...,...,...,...,...,...,...,...,...
220,120,2564.42,1720.22,4387.03,1681.14,591.167,4076.24,1
221,121,2562.48,1711.98,4394.47,1679.87,582.125,4081.64,1
222,122,2560.53,1703.65,4402.07,1678.34,573.083,4087.00,1
223,123,2558.47,1695.52,4409.49,1676.52,564.042,4092.30,1


In [12]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [13]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [14]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [15]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P185/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/analysis'], returncode=0)